# Object Oriented Programming (Classes)

In [11]:
D = {'a': 1, 'b': 2}

In [12]:
D['a']

1

In [13]:
D['c']

KeyError: 'c'

### Making Your Own Default Dictionary

Let's make our own default dictionaries.

There are two concepts we need

  - A `class` is a template for a new data type.  It contains inforamtion on what data is needed to constrict the data type, how to store the data internally, and what algorithms can be applied to the data type.
  - An instance of a class is a concrete object of the new data type.
  
A class is a recipe for constructing instances of that class.

#### Example of a Class: defaultdict

`defaultdict` is a class

In [14]:
from collections import defaultdict
from inspect import isclass
isclass(defaultdict)

True

Using the class `defaultdict` as a function creates an instance of that class.

In [66]:
D = defaultdict(lambda: 0, {'a': 1, 'b': 2})
isinstance(D, defaultdict)

True

In [68]:
print(D)

defaultdict(<function <lambda> at 0x1048fac80>, {'a': 1, 'b': 2})


We usually abbreviate the phrase

> `D` is an instance of class `defaultdict`.

as

> `D` is a `defaultdict`.

**Let's try to create our own implementation of a default dictionary.**

The first step is to determine what data we need to store.  In this case it's pretty easy, we need

  - The underlying dictionary that we are going to attempt lokups into.
  - The default action to take when a lookup fails.

Let's mimic the way python's built in default dict works.  We need to add some functionality to **supply and then store** both of these data elements when we create an instance of the class.  This is done using a special *method*, `__init__`.

**Note:** `__init__` is pronounced *dunder-in-it*.

In [55]:
class MyDefaultDict(object):
    """A personal implementation of a default dictionary."""
    
    def __init__(self, default, dictionary):
        self.default = default
        self.dictionary = dictionary

There's a lot of new concepts in this code, but let's first see how it works.

In [56]:
MD = MyDefaultDict(lambda: 0, {'a': 1, 'b': 2})
print(MD.default)
print(MD.default())
print(MD.dictionary)

<function <lambda> at 0x1048fa6e0>
0
{'a': 1, 'b': 2}


When we use a class, it is to create *instances of that class*, which we then work with.  We very rarely work with the class directly, and we will often be working with more than one instance of a single class.

In [57]:
MD = MyDefaultDict(lambda: 0, {'a': 1, 'b': 2})
MD2 = MyDefaultDict(lambda: 1, {'a': 2, 'b': 3, 'c': 5})

print(MD.dictionary)
print(MD2.dictionary)

{'a': 1, 'b': 2}
{'a': 2, 'c': 5, 'b': 3}


Inside of the code defining a class, `self` represents the instance of the class we are manipulating.

A statement like

```
self.default = default
```

creates what is known as an *instance varaible* or *instance data*.  In this specific line, we attach the `default` function to the current instance of the class.

In this way, once created, each instance of `MyDefaultDict` stores both `default` and `dictionary` data.

In [58]:
MD['a']

TypeError: 'MyDefaultDict' object has no attribute '__getitem__'

#### Addding Methods to Manipulate Data in a Class

Let's implement `__getitem__` and `__setitem__`, which will allow us to index into instances of our class like this

```
MD['a']
# Means the same thing as MD.__getitem__('a')

MD['c'] = 3
# Means the same thing as MD.__setitem__('c', 3)
```

As a first attempt, let's ignore our goal of adding default behaviour, we can add that later on down the line.

In [59]:
class MyDefaultDict(object):
    """A personal implementation of a default dictionary."""
    
    def __init__(self, default, dictionary):
        self.default = default
        self.dictionary = dictionary
        
    def __getitem__(self, key):
        return self.dictionary[key]
    
    def __setitem__(self, key, value):
        self.dictionary[key] = value

Let's test it out.

In [60]:
MD = MyDefaultDict(lambda: 0, {'a': 1, 'b': 2})

print(MD['a'])
print(MD['b'])

MD['c'] = 3

print(MD.dictionary)

1
2
{'a': 1, 'c': 3, 'b': 2}


In [63]:
type(MD.dictionary)

dict

In [62]:
MD['d']

KeyError: 'd'

#### Adding the Special Default Behaviour

In [69]:
class MyDefaultDict(object):
    """A personal implementation of a default dictionary."""
    
    def __init__(self, default, dictionary):
        self.default = default
        self.dictionary = dictionary
        
    def __getitem__(self, key):
        if key in self.dictionary:
            return self.dictionary[key]
        else:
            self.dictionary[key] = self.default()
            return self.dictionary[key]
    
    def __setitem__(self, key, value):
        self.dictionary[key] = value
        
    def __eq__(self, other):
        self.dictionary == other.dictionary

In [64]:
MD = MyDefaultDict(lambda: 0, {'a': 1, 'b': 2})

print(MD['a'])
print(MD['b'])
print(MD['c'])
print(MD.dictionary)

1
2
0
{'a': 1, 'c': 0, 'b': 2}


#### Adding Other Dict-y Things

A few things that should work for dictionaries still don't work for our new datatype

In [79]:
len(MD)

TypeError: object of type 'MyDefaultDict' has no len()

Additionally, code like

```
'c' in MD
```

and

```
for key in MD:
    print key, MD[key]
```

will cause an infinite loop, due to a design error in python itself.

Let's fix that with more magic methods.

In [70]:
class MyDefaultDict(object):
    """A personal implementation of a default dictionary."""
    
    def __init__(self, default, dictionary):
        self.default = default
        self.dictionary = dictionary
        
    def __getitem__(self, key):
        if key in self.dictionary:
            return self.dictionary[key]
        else:
            return self.default()
    
    def __setitem__(self, key, value):
        self.dictionary[key] = value
        
    def __len__(self):
        return len(self.dictionary)
        
    def __contains__(self, key):
        return key in self.dictionary
    
    def __iter__(self):
        for key in self.dictionary:
            yield key

In [71]:
MD = MyDefaultDict(lambda: 0, {'a': 1, 'b': 2})

In [74]:
print('a' in MD)

True


In [75]:
for key in MD:
    print key, MD[key]

a 1
b 2


In [77]:
MD['c'] = 100
len(MD)

3

#### Non-Magic Methods

It's worth mentioning that not all methods are magic.  Here is a class that represents a simple quadratic polynomial, and has an `evaluate` method, which plugs a number into the polynomial.

In [79]:
class QuadraticPolynomial(object):
    """A class representing a polynomial like:
    
        a_0 + a_1 x + a_2 x^2
    """
    
    def __init__(self, a0, a1, a2):
        self.coefficients = (a0, a1, a2)
        
    def evaluate(self, x):
        a0, a1, a2 = self.coefficients
        return a2*x*x + a1*x + a0

In [80]:
QuadraticPolynomial(1, 1, 1).evaluate(1)

3

In [82]:
x = QuadraticPolynomial(1, 1, 1)
print(x)

#### Excercises

1. Implement a `__repr__` method that represents the `QuadraticPolynomial` object as a string.  This string should, if interpreted as code, evaluate to the object itself, so should return something like `QuadraticPolynomial(1, 2, 3)`.

1. Implement a `__str__` method that implements a `QuadraticPolynomial` as a string, but this time in a more human readable way.  So something like `x^2 + 2x + 3`.

1. Implement the `__add__` magic method to allow something like `QuadraticPolynomial(1, 1, 1) + QuadraticPolynomial(1, 0, 1)`.  The new method should *return* another `QuadraticPolynomial`.

1. Implement the `__sub__` magic method to allow subtraction.

1. Refactor `__add__` and `__sub__` to remove code duplication.

1. Should `QuadraticPolynomial` implement `__getitem__` or `__setitem__`?  If so, what should they do?  What do we have to change inside our class to support this?

1. Write a class `LinearPolynomial`.  Add a method `differentiate` to `QuadraticPolynomial` that returns a `LinearPolynomial`.

1. Suppose we create a new `QuadraticPolynomial` like `QuadraticPolynomial(1, 1, 0)`.  Is this really a `QuadraticPolynomial`?  What should it be?  How can you resolve this weird inconsistency in data types?

In [100]:
class QuadraticPolynomial(object):
    """A class representing a polynomial like:
    
        a_0 + a_1 x + a_2 x^2
    """
    
    def __init__(self, a0, a1, a2):
        self.coefficients = (a0, a1, a2)
        
    def evaluate(self, x):
        a0, a1, a2 = self.coefficients
        return a2*x*x + a1*x + a0
    
    def __repr__(self):
        return "QuadraticPolynomial({}, {}, {})".format(
            self.coefficients[0],
            self.coefficients[1],
            self.coefficients[2]
        )
    
    def __str__(self):
        return "{}x^2 + {}x + {}".format(
            self.coefficients[2],
            self.coefficients[1],
            self.coefficients[0]
        )
    
    def __add__(self, other):
        a0, a1, a2 = self.coefficients
        b0, b1, b2 = other.coefficients
        return QuadraticPolynomial(a0 + b0, a1 + b1, a2 + b2)
    
    def derivative(self):
        a0, a1, a2 = self.coefficients
        return LinearPolynomial(a1, 2*a2)

In [104]:
class LinearPolynomial(object):
    
    def __init__(self, a0, a1):
        self.coefficients = (a0, a1)
        
    def __repr__(self):
        return "LinearPolynomial({}, {})".format(
           self.coefficients[0], self.coefficients[1]
        )

In [89]:
x = QuadraticPolynomial(1, 0, 1)
print(x)

1x^2 + 0x + 1


In [92]:
print(QuadraticPolynomial(1, 1, 1) + QuadraticPolynomial(1, -1, 2))

3x^2 + 0x + 2


In [105]:
print(QuadraticPolynomial(1, -1, 2).derivative())

LinearPolynomial(-1, 4)


### Classes and Functions

We now have two powerful methods to organize our programs, **functions** and **classes**.  It seems important to discuss when we should use one or the other.

This is especially important, because in some sense the concepts are interchangable.

#### Functions Can Be Represented as Classes

Consider the following function, which sums up the elements in a list that satisfy some condition

In [106]:
def sum_elements_in_list_satisfying_condition(lst, condition):
    s = 0
    for elem in lst:
        if condition(elem):
            s += elem
    return s

In [107]:
sum_elements_in_list_satisfying_condition([0, 1, -1, 2, -1, 3], lambda x: x >= 0)

6

In [108]:
sum_elements_in_list_satisfying_condition([0, 1, -1, 2, -2, 3], lambda x: x <= 0)

-3

This function could be instead represented in our program as a class with a *one method interface*.

In [109]:
class ListConditionAdder(object):
    
    def __init__(self, lst, condition):
        self.lst = lst
        self.condition = condition
        
    def run(self):
        s = 0
        for elem in self.lst:
            if self.condition(elem):
                s += elem
        return s

In [111]:
lca = ListConditionAdder([0, 1, -1, 2, -1, 3], lambda x: x >= 0)
lca.run()

6

In [112]:
lca = ListConditionAdder([0, 1, -1, 2, -1, 3], lambda x: x <= 0)
lca.run()

-2

This seems like a silly thing to do, but consider the case where where:

  - We have few conditions, that we want to use many times.
  - We have many, many lists, that we want to sum with only a few conditions.
  
In this case, creating objects makes some sense.  This class makes the intention very clear:

In [126]:
class ListConditionAdder(object):
    
    def __init__(self, condition):
        self.condition = condition
        
    def run(self, lst):
        s = 0
        for elem in lst:
            if self.condition(elem):
                s += elem
        return s

In [127]:
lca = ListConditionAdder(lambda x: x >= 0)

In [128]:
lca.run([1, 1, 1])

3

In [129]:
lca.run([1, -1, 1])

2

In [130]:
lca.run([2, -1, -2])

2

#### Classes can be Represented as Functions

Classes can be represented as functions, though this is not well supported in python.

In [120]:
def new_quadratic_polynomial(a, b, c):
    
    coefficients = (a, b, c)
    
    def eval(x):
        return coefficients[0]*x*x + coefficients[1]*x + coefficients[2]
    
    return {
        "coefficients": coefficients,
        "eval": eval
    }

In [121]:
qp = new_quadratic_polynomial(1, 1, 1)

In [122]:
qp["coefficients"]

(1, 1, 1)

In [123]:
qp["eval"](1)

3

When used in this way, a function is called a **closure**.

  - The local varaibles of the function become the attributes of the object.
  - The dictionary you return becomes the object.
  - The function itself is the type of the object.
  
Some languages do not rely on classes at all, and instead use this pattern to provide object style behaviour.  Scheme, Clojure, and Javascript are examples.

## The Three Pillars

Now we can summarize the three pillars of object oriented design:

- **Polymorphism**: Different examples of similar objects can have different algorithms that *perform the same task*, programmers should not have to explicitly call out the algorithm, it should be chosen based on the data type.

For example, we should *not* have multiple functions/methods like

```
len_list
len_tuple
len_dict
len_polynomial
len_some_other_datatype
```

Instead the programmer should have one exposed function, `len`, and the *algorithm* should be selected based on the data type.

  - **Encapsulation**: The way we *actually* store data is not important, we should provide an interface to interact with it that hides tricky or irrelevent details, and this is what the downstream programmers should use.
  
Recall our discussion about how `dict`s work (hashing, there's a list underneath, blah, blah).  You can *use* a `dict` without knowing all this detail (and many people do!).  This is the essense of encpsulation, python provides a flexible interface for dictionaries, and we can do pretty much all we need to do by working with the interface!

The final pillar is... controversial:

  - **Inheritence**: A program should have a heirchy of data types, where the childeren *steal* algorithms from the parent.

In [8]:
class Animal(object):
    
    def speak(self):
        return "I cannot speak."
    
    
class Dog(Animal):
    
    def speak(self):
        return "WOOF!"
    

class Fish(Animal):
    pass

In [124]:
dog = Dog()
dog.speak()

'WOOF!'

In [125]:
fish = Fish()
fish.speak()

'I cannot speak.'

Both `Dog` and `Fish` *inherit* from the parent class `Animal`.  The `Animal` class provides default `speak` behaviour, which `Dog` overrides, but `Fish` does not!